# Проект большие языковые модели

### Подготовка окружения

In [6]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast, LlamaConfig, LlamaForCausalLM, get_linear_schedule_with_warmup
from torch.optim import AdamW 
import os
import torch
import warnings
from torch.utils.data import DataLoader
import regex as re
import numpy as np
import random
from pathlib import Path
from datasets import Dataset
import unicodedata
from collections import Counter
from tqdm.notebook import tqdm

In [7]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True

seed_everything(42)
warnings.filterwarnings('ignore')

In [26]:
DATA_DIR = Path("RussianNovels/corpus")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 3000
CYRILLIC_RE = re.compile(r"^[\p{IsCyrillic}\s0-9.,!?—–:;\"'()«»…\-]+$")
BATCH_SIZE = 4
MAX_EPOCHS = 10
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 0.01
GRAD_CLIP = 1.0
SAVE_PATH = "best_model.pt"

test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
] 

print(f"{DEVICE=}")

DEVICE='cpu'


In [9]:
!git clone https://github.com/JoannaBy/RussianNovels.git
!git clone https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru

fatal: destination path 'RussianNovels' already exists and is not an empty directory.
fatal: destination path 'alpaca-cleaned-ru' already exists and is not an empty directory.


## Pretrain

### Подготовка корпуска текстов

In [10]:
def collect_texts(root: Path):
    texts = []
    for path in root.rglob("*"):
        if path.suffix in {".txt"}:
            try:
                texts.append(path.read_text(encoding="utf-8"))
            except Exception:
                pass
    return texts

raw_texts = collect_texts(DATA_DIR)
raw_texts = list(set(raw_texts))
print(f"Файлов в корпусе: {len(raw_texts)}")

Файлов в корпусе: 107


In [ ]:
def filter_cyrillic_sentences(text: str):
    '''фильтруем предложения с некириллическими символами'''
    sentences = re.split(r"[.!?]+", text)
    return [
        s.strip()
        for s in sentences
        if s.strip() and CYRILLIC_RE.match(s.strip())
    ]

def normalize_punctuation(text: str) -> str:
    '''нормализуемы пунктуацию'''
    text = re.sub(r"[!?]{2,}", r"\1", text)
    text = re.sub(r"\.{3,}", "…", text)
    text = re.sub(r",,{2,}", ",", text)
    text = re.sub(r"--{2,}", "-", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'[«»]', '"', text)
    return text.strip()    

clean_sentences = []

for text in raw_texts:
    sentences = filter_cyrillic_sentences(text)
    sentences = [normalize_punctuation(s) for s in sentences]
    clean_sentences.extend(sentences)

print(f"Предложений после очистки: {len(clean_sentences)}")

Предложений после очистки: 523962


In [12]:
print("Проверка состава оставшихся спец символов")
def is_letter_or_digit(ch: str) -> bool:
    cat = unicodedata.category(ch)
    return cat.startswith("L") or cat.startswith("N")

counter = Counter()

for sent in clean_sentences:
    for ch in sent:
        if not is_letter_or_digit(ch):
            counter[ch] += 1


non_alnum = counter.most_common()
print(f"symb | freg    | name")
for ch, freq in non_alnum[:30]:
    name = unicodedata.name(ch, "UNKNOWN")
    print(f"'{ch}'  | {freq:>7} | {name}")

Проверка состава оставшихся спец символов
symb | freg    | name
' '  | 5716309 | SPACE
','  |  853248 | COMMA
'-'  |  410564 | HYPHEN-MINUS
'"'  |   49998 | QUOTATION MARK
':'  |   44922 | COLON
';'  |   32563 | SEMICOLON
'–'  |   24641 | EN DASH
')'  |    7897 | RIGHT PARENTHESIS
'('  |    7337 | LEFT PARENTHESIS
'—'  |    4435 | EM DASH
'…'  |    2988 | HORIZONTAL ELLIPSIS
'''  |    1152 | APOSTROPHE


In [13]:
# разбиение на чанки
chunks = []
current = ""

for sent in clean_sentences:
    max_chars = 2000  # ~512 токенов
    if len(current) + len(sent) < max_chars:
        current += " " + sent
    else:
        chunks.append(current.strip())
        current = sent

if current:
    chunks.append(current.strip())

print(f"Чанков: {len(chunks)}")
print(f"чанк  0: {chunks[0]}")
print(f"чанк 10: {chunks[10]}")


Чанков: 20114
чанк  0: Он был уже в отставке и жил со старушкой-матерью в Петербурге, на Литейной, где нанимал очень хорошую квартиру во втором этаже, ездил в Михайловский театр, обедал нередко в Английском клубе, имел несчетное множество тетушек и дядюшек, значительное имение в Тульской губернии и такое же точно значительное количество -- долгу Что делать В молодости он служил в гусарах позашалился, проигрался; мать была в отчаянии, а тетушки О них нечего и говорить Особливо одна тетушка, Елена Павловна; эта решительно была уже вне себя По всему городу, то есть по всем знакомым домам, точно тревогу били тетушки, и везде слышно было по секрету: "Бедная сестра Несчастная женщина Повеса Погубил бедную мать Если, б был отец, конечно, уж не то бы было", -- и тому подобное При таком множестве тетушек, конечно, было так же много и кузин; кузины также шумели и даже плакали; но они говорили: "Это просто несчастие; Евгений превосходный человек; святая, небесная душа Он был завлечен, но он испра

### Обучение токенизатора

In [14]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# Нормализация текста
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
# разбиение по пробелам 
tokenizer.pre_tokenizer = Whitespace() 
trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]"] 
    )
tokenizer.train_from_iterator(chunks, trainer)
# (BOS/EOS) 
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    pair="[BOS] $A [EOS] $B:1 [EOS]:1",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]")), 
        ], 
        )
# Сохранение
tokenizer.save("bpe_ru_3k.json")

In [15]:
encoded = tokenizer.encode("Пример текста для проверки токенизатора")
print(encoded.tokens)
print(encoded.ids)


['[BOS]', 'пример', 'те', 'к', 'ста', 'для', 'про', 'вер', 'ки', 'то', 'ке', 'ни', 'за', 'тора', '[EOS]']
[2, 1905, 80, 31, 124, 309, 107, 171, 101, 58, 221, 70, 81, 1681, 3]


### Сборка датасета

In [16]:

hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="bpe_ru_3k.json",
    bos_token="[BOS]",
    eos_token="[EOS]",
    unk_token="[UNK]",
    pad_token="[PAD]",
)

def tokenize(batch):
    return hf_tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def add_labels(batch):
    labels = []
    for ids, mask in zip(batch["input_ids"], batch["attention_mask"]):
        l = ids.copy()
        l = [
            token if m == 1 else -100
            for token, m in zip(l, mask)
        ]
        labels.append(l)

    batch["labels"] = labels
    return batch


dataset = Dataset.from_dict({"text": chunks})

dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"],
).map(add_labels, batched=True)

dataset.set_format(type="torch")

Map:   0%|          | 0/20114 [00:00<?, ? examples/s]

Map:   0%|          | 0/20114 [00:00<?, ? examples/s]

### Подготовка модели LlamaForCausalLM

In [17]:
config = LlamaConfig(
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=2048,
    rms_norm_eps=1e-6,
    hidden_act="silu",
    tie_word_embeddings=True,
    attention_bias=False,
    rope_theta=10000.0,
    use_cache=False,     
)
config.vocab_size = max(hf_tokenizer.get_vocab().values()) + 1
config.pad_token_id = hf_tokenizer.pad_token_id
config.bos_token_id = hf_tokenizer.bos_token_id
config.eos_token_id = hf_tokenizer.eos_token_id

model = LlamaForCausalLM(config).to(DEVICE)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Обучаемых параметров: {num_params:,}")

Обучаемых параметров: 128,934,912


In [18]:
print("Проверим синхронизацию модели и текенизатора")
assert model.config.vocab_size == max(hf_tokenizer.get_vocab().values()) + 1
assert model.config.pad_token_id == hf_tokenizer.pad_token_id
assert model.config.eos_token_id == hf_tokenizer.eos_token_id
print("ок")

Проверим синхронизацию модели и текенизатора
ок


In [19]:
print("Sanity-check начальной модели на инференсе")
model.eval()

prompt = "Привет"
inputs = hf_tokenizer(
    prompt,
    return_tensors="pt",
    return_token_type_ids=False
).to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=False,
        eos_token_id=hf_tokenizer.eos_token_id,
    )

print(hf_tokenizer.decode(out[0], skip_special_tokens=True))


Sanity-check начальной модели на инференсе
при вет сразу сразу сразу сразу сразу сразу хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол хол


In [20]:
# оптимизатор
def get_optim():
    return AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

optimizer = get_optim()

In [ ]:
print("Сheck сходимости модели при обучение на малом датасете (2 предожения)")
tiny_dataset = dataset.select(range(2))
tiny_loader = DataLoader(tiny_dataset, batch_size=1, shuffle=True)

model.train()

progress_bar = tqdm(
        range(201),
        desc=f"train check",
    )
best_loss = float("inf")

for step in progress_bar:
    batch = next(iter(tiny_loader))
    batch = {k: v.to(model.device) for k, v in batch.items()}

    loss = model(**batch).loss
    progress_bar.set_description(f"train check | loss: {loss:.2f}")
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if step> 0 and step % 20 == 0:
        progress_bar.write(f"epoch: {step:>3} loss: {loss.item():.2f}")
        if loss.item() < best_loss:
            best_loss = loss.item()
            torch.save(model.state_dict(), SAVE_PATH)
            progress_bar.write(f"best model saved with {best_loss=:.2f}")

Сheck сходимости модели при обучение на малом датасете (2 предожения)


train check:   0%|          | 0/201 [00:00<?, ?it/s]

best model saved with best_loss=8.25
best model saved with best_loss=7.32
best model saved with best_loss=6.62
best model saved with best_loss=6.26
best model saved with best_loss=5.96
best model saved with best_loss=5.94
best model saved with best_loss=5.56
best model saved with best_loss=5.56
best model saved with best_loss=5.52
epoch:  20 loss: 5.56
best model saved with best_loss=5.52
best model saved with best_loss=5.47
best model saved with best_loss=5.43
best model saved with best_loss=5.38
best model saved with best_loss=5.36
best model saved with best_loss=5.27
best model saved with best_loss=5.21
epoch:  40 loss: 5.13
best model saved with best_loss=5.13
best model saved with best_loss=5.11
best model saved with best_loss=5.08
best model saved with best_loss=5.05
best model saved with best_loss=5.00
best model saved with best_loss=4.98
best model saved with best_loss=4.97
best model saved with best_loss=4.95
best model saved with best_loss=4.92
epoch:  60 loss: 5.42
best mode

In [ ]:
print("Сheck инференса модели после обучения на малом датасете")
def check_inference(model, prompts: list):
    model.eval()
    for pr in prompts:
        inputs = hf_tokenizer(pr, return_tensors="pt").to(model.device)
        inputs.pop("token_type_ids", None)

        out = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True, 
        eos_token_id=hf_tokenizer.eos_token_id,
    )
        print("input:  ",  pr)
        print("output: ", hf_tokenizer.decode(out[0], skip_special_tokens=True))
        print()


check_inference(model, test_prompts[:2])

Сheck инференса модели после обучения на малом датасете
input:   Все мысли, которые имеют огромные последствия
output:  все мысли , которые име ют огром ные послед ствия е " , - уже ли и у - за на не не , он , же рался се в не в не я , вы , но , и - на ра , я что -- в -- -- , , , е не м бе , а и
input:   Сила войска зависит от его духа
output:  сила вои ска зави си т от его ду ха ли а му в , , ра от ром он , на же на ра то про сти , теперь , по - я с от раз ли и с в было , , это , имел вы -- это не , о , где как в , не ку


### Обучение модели LlamaForCausalLM

In [23]:
train_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
model.to(DEVICE)
model.train()

optimizer = get_optim()

total_steps = len(train_loader) * MAX_EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps // 10, 
    num_training_steps=total_steps
)

best_loss = float("inf")

for epoch in range(MAX_EPOCHS):

    progress_bar = tqdm(train_loader)

    total_loss = 0.0
    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        loss = outputs.loss

        progress_bar.set_description(
            f"train epoch {str(epoch+1):<2} step {str(step+1):<3} loss: {loss.item():.2f}"
        )
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)

        # Optimizer step
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        if step>0 and step % 50 == 0:
            progress_bar.write(f"epoch {epoch+1} step {step+1} loss: {loss.item():.2f}")

    avg_loss = total_loss / len(train_loader)

    progress_bar.write(f"epoch {epoch+1} finished. avg_loss: {avg_loss:.2f}")
    
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), SAVE_PATH)
        progress_bar.write(f"best model saved with {best_loss=:.2f}")

  0%|          | 0/5029 [00:00<?, ?it/s]

epoch 1 step 51 loss: 8.01


KeyboardInterrupt: 